In [4]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
persist_directory = '/workspaces/Medical_Chat/WHYembedding/精神科/vector_db'

def init_db(persist_path = None, 
            embedding = None):
    vectordb = Chroma(
        persist_directory=persist_path,
        embedding_function=embedding
    )
    # vectordb.persist() 
    vectordb.persist()
    return vectordb

# create the vectordb object
vectordb = init_db(persist_path=persist_directory, embedding=embedding) 
print(type(vectordb))
print(f"向量库中存储的数量：{vectordb._collection.count()}")
question = "临床?"
docs = vectordb.similarity_search(question, k=3)
print(f"检索到的内容数：{len(docs)}")
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n***************************************************\n")


/opt/conda/envs/MChat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'langchain_community.vectorstores.chroma.Chroma'>
向量库中存储的数量：10
检索到的内容数：3
检索到的第0个内容: 
 目录21一、流行病学233二、病因233三、临床表现234四、病程与预后 236五、诊断与鉴别诊断236六、预防与治疗237第三节孤独症谱系障碍239一、概述239二、流行病学239三、临床表现239四、诊断与鉴别诊断241五、病程及预后 242六、干预和治疗242第四节注意缺陷多动性障碍243一、流行病学244二、病因和病理机制244三、临床表现244四、诊断与鉴别诊断245五、病程和预后246六、治疗和干预247第五节抽动障碍 249一、流行病学 249二、病因和病理机制249三、临床表现249四、诊断与鉴别诊断 250五、病程与预后251六、治疗 251第六节特定性学习障碍252第十三章精神分裂症 256第一节概述 256第二节病因与发病机制257一、遗传因素257二、神经病理学及大脑结构的异常 259三、神经生化方面的异常261四、神经发育不良假说261五、子宫内感染与产伤 262六、社会心理因素263第三节临床表现263第四节诊断与鉴别诊断 264一、诊断标准264二、鉴别诊断 266三、实验室检查 266
***************************************************
检索到的第1个内容: 
 24■目录四、病程和预后 331五、病因和发病机制研究331六、治疗331七、预防和康复 331第四节社交焦虑障碍331一、流行病学 332二、临床表现 332三、诊断与鉴别诊断 332四、病程和预后333五、病因和发病机制研究 333六、治疗 333七、预防和康复334第五节特定恐惧症334一、流行病学 335二、临床表现 335三、诊断与鉴别诊断 335四、病程和预后335五、病因和发病机制研究 336六、治疗 336七、预防和康复336第六节广泛性焦虑障碍337一、流行病学 337二、临床表现 337三、诊断与鉴别诊断337四、病程和预后338五、病因和发病机制研究339六、治疗 339七、预防和康复340第七节分离焦虑障碍340一、流行病学 340二、临床表现341三、诊断与鉴别诊断 341四、病程和预后 341五、病因和发病机制研究

### 创建一个llm

In [2]:
from test_qianfan_llm import QianFanLLM
llm = QianFanLLM(model_name="Yi-34B-Chat", 
                   api_key="HnngI6IOybeRJyLgvtowIRUt", 
                   api_secret="Vx7tFMIdGDrouCsyBRuChmbNBHhlMM2e")
result = llm.invoke("你好")
print(result)

你好！我是一个人工智能助手，随时准备帮助你解答问题或提供信息。请问有什么我可以帮助你的？


In [5]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=False,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [47]:
question_1 = "什么是精神科疾病"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
精神科疾病是指一组影响心理和行为的疾病，它们可以导致情绪、思维、行为和社交功能的异常。精神科疾病包括抑郁症、焦虑症、精神分裂症、双相情感障碍、强迫症、创伤后应激障碍等。这些疾病可能由遗传、生物化学、心理社会因素等多种原因引起，通常需要专业的心理治疗、药物治疗或两者结合来管理。精神科疾病的严重程度可以从轻微的困扰到严重的残疾不等，它们可能对患者的日常生活和福祉产生显著影响。谢谢你的提问！


In [48]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.invoke(prompt_template)

'精神科疾病是指一组影响个体情绪、思维、行为和心理状态的疾病。这些疾病可能包括抑郁症、焦虑症、双相情感障碍、精神分裂症、强迫症、创伤后应激障碍等。精神科疾病的症状和严重程度可以有很大差异，从轻微的困扰到严重的功能障碍不等。\n\n精神科疾病的诊断通常需要由经过专业培训的精神科医生进行，他们通过病史、心理评估和有时包括脑部影像学检查或实验室检查在内的医学检查来做出诊断。治疗方法可能包括药物治疗、心理治疗、支持性治疗和社会干预。\n\n精神科疾病是一个广泛的领域，它不仅关注疾病的治疗，还包括预防、康复和提高患者生活质量的努力。随着对大脑和心理健康的了解不断深入，精神科领域也在不断发展新的治疗方法和更有效的干预措施。'

# 历史对话功能

In [49]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
)
question = "什么是精神科疾病"
result = qa({"question": question})
print(result['answer'])

question = "精神科疾病有哪些"
result = qa({"question": question})
print(result['answer'])

精神科疾病是指一组影响心理和行为的疾病，它们可以导致个体在情感、思维、行为、认知和社会功能等方面出现异常。精神科疾病包括多种类型，如抑郁症、焦虑症、精神分裂症、双相情感障碍、强迫症、创伤后应激障碍等。这些疾病可能由遗传、生物化学、心理社会因素等多种原因引起，并且可以对个体的日常生活和福祉产生显著影响。精神科疾病的诊断通常需要由经过专业训练的精神科医生进行，他们通过详细的病史、心理评估和必要的辅助检查来确定患者的具体诊断。治疗方法包括药物治疗、心理治疗、社会支持和其他康复措施。
根据您提供的信息，精神科疾病可以分为以下几类：

1. 器质性精神障碍（Organic Mental Disorders）：这类疾病与大脑或身体的生理变化有关，通常由脑损伤、感染、营养不良或其他身体疾病引起。

2. 使用精神活性物质所致的精神或行为障碍（Substance-Induced Disorders）：这类障碍是由于使用酒精、药物或其他精神活性物质引起的。

3. 精神分裂症、分裂型障碍和妄想性障碍（Schizophrenia, Schizotypal, and Delusional Disorders）：这些疾病通常涉及严重的妄想和幻觉，以及思维、情感和行为的紊乱。

4. 心境（情感）障碍（Mood Disorders）：这类障碍包括抑郁症、躁狂症、双相情感障碍等，涉及情绪的极端波动。

5. 神经症性、应激相关的及躯体形式障碍（Neurotic, Stress-Related, and Somatic Symptom Disorders）：这类障碍包括焦虑症、创伤后应激障碍、躯体症状障碍等，通常与压力和心理因素有关。

6. 伴有生理素乱及驱体因素的行为综合征（Disorders with Psychophysiological and Somatic Factors）：这类障碍涉及身体症状和心理因素的相互作用，如进食障碍、睡眠障碍等。

7. 成人人格与行为障碍（Personality and Behavioral Disorders in Adults）：这类障碍包括边缘型人格障碍、自恋型人格障碍等，涉及长期的性格特征和行为模式。

8. 精神发育迟滞（Mental Retardation）：现在通常称为智力残疾，是指在发育期间由于遗传、环境或医疗原因导致的智力功能显著低于